In [ ]:
# Import python packages
import streamlit as st
import pandas as pd
# Make sure 'count' is included in the import statement
from snowflake.snowpark.functions import col, count, lit, sum
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.ensemble import IsolationForest


# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()


In [ ]:
import warnings
warnings.filterwarnings(
    "ignore",
    message="Could not find the number of physical cores for the following reason:*",
    category=UserWarning,
    module="joblib"
)


In [ ]:
# query github evets wide
gh_events_wide_table_name = "GITHUB_EVENTS_WIDE"
gh_events_wide_df = session.table(gh_events_wide_table_name)
global_events_table_name = "GLOBAL_EVENTS"
global_events_df = session.table(global_events_table_name)
actor_table_name = "ACTOR"
actor_df = session.table(actor_table_name)
repo_table_name = "REPO"
repo_df = session.table(repo_table_name)
org_table_name = "ORG"
org_df = session.table(org_table_name)
# concatenate all the dfs in a list to generalize data analysisimport warnings
names_fds = [(gh_events_wide_table_name, gh_events_wide_df), 
       (global_events_table_name, global_events_df),
       (actor_table_name, actor_df),
       (repo_table_name, repo_df),
       (org_table_name, org_df)
      ]

Data exploration

In [ ]:
# for each df print desc
for name, df in names_fds:
    print(f"Description: {name} table:")
    print(df.describe().show())
    print('columns/features schema:')
    print(df.printSchema())
    print('number of rows:')
    print(df.count())

    print('===================================================')
    

It seems GLOBAL_EVENTS and ACTOR are the most informative data. Focus on them:
- GLOBAL_EVENTS:
    - ID: Event id. non-informative
    - TYPE: closed vocabulary, define the action. 
    - ACTOR_ID: Used to merge with Actor.
    - REPO_ID: non-informative in this context.
    - ORG_ID: Id, could be used as a boolean when is missing.
    - PUBLIC: All are TRUE, non-informative.
    - CREATED_AT: Datetime, we could use the day of the week o hour.

- ACTOR
    - ID: used to connect with GLOBAL_EVENTS.
    - LOGIN: custom name
    - GRAVATAR_ID: empty, no-informative.
    - URL: All distinct, non-informative.
    - AVATAR_URL: All distinct, non-informative

Preprocess

In [ ]:
# merge the df global events and actor
actor_event_df = global_events_df.join(actor_df, global_events_df["actor_id"] == actor_df["id"], how="left")
actor_event_df.to_pandas().head()

In [ ]:
# Day of the week, hour
# count event types
# org boolean
actor_counts_df = actor_event_df.group_by("ACTOR_ID", "TYPE").agg(count(lit(1)).alias("EVENT_COUNT"))
actor_counts_df.to_pandas().head(30)

In [ ]:
# get the unique tpyes 
distinct_types = [row["TYPE"] for row in actor_counts_df.select("TYPE").distinct().collect()]
# Pivot by TYPE the table
actor_counts_df = actor_counts_df.pivot(
    pivot_col = "TYPE",        
    values = distinct_types    
).agg(
    sum("EVENT_COUNT")         
)
actor_counts_df.to_pandas().head(30)

In [ ]:
# fill miisng values with 0
actor_counts_df = actor_counts_df.na.fill(0)
actor_counts_df.to_pandas().head(30)

In [ ]:
# remove ACTOR_ID column, not longer needed
actor_counts_df.drop('ACTOR_ID')

In [ ]:
# sample the data 
actor_counts_df_pandas_df = actor_counts_df.to_pandas()
actor_counts_df_pandas_df = actor_counts_df_pandas_df.sample(200000)

In [ ]:
# Apply min max normalization, KMEANs and DBSCAN are sensitive different scales
scaler = StandardScaler()
actor_counts_df_pandas_scaled = scaler.fit_transform(actor_counts_df_pandas_df)
actor_counts_df_pandas_scaled

Train

In [ ]:
kmeans = KMeans(n_clusters=8, random_state=42, n_init='auto') # n_init='auto' is default in newer sklearn
kmeans.fit(actor_counts_df_pandas_scaled)
cluster_labels = kmeans.labels_

In [ ]:
cluster_labels

Evaluation

In [ ]:
# 4. Calculate Inertia for range of k
inertia_values = {}
k_range = range(2, 50)

for k in k_range:
    kmeans = KMeans(
        n_clusters=k,
        random_state=42,
        n_init='auto'
    )
    kmeans.fit(actor_counts_df_pandas_scaled)
    inertia_values[k] = kmeans.inertia_


In [ ]:
k_values = sorted(inertia_values.keys())
inertia_vals = [inertia_values[k] for k in k_values] 

plt.figure(figsize=(10, 6)) 


plt.plot(
    k_values,
    inertia_vals,
    marker='o',         
    linestyle='-',      
    color='b'           
)


plt.xlabel("Number of Clusters (k)")
plt.ylabel("Inertia")
plt.title("Elbow Method for Optimal k")

plt.xticks(k_values) 
plt.grid(True, linestyle='--', alpha=0.7) 

plt.show()

Train Outlier Detection


Used to indentify bots/smap based on number of actions 

In [ ]:
iso_forest = IsolationForest(n_estimators=200, contamination='auto', random_state=42)
iso_forest.fit(actor_counts_df_pandas_scaled)

# Predict anomalies (-1 for outliers, 1 for inliers)
outliers = iso_forest.predict(actor_counts_df_pandas_scaled)

# Create masks for inliers
iso_outliers_mask = outliers == -1

# Remove outliers from train and test sets (X and y)
outliers_index = actor_counts_df_pandas_df.loc[iso_outliers_mask]

In [ ]:

print(f"there are: {outliers_index.shape[0]} outliers")

In [ ]:
outliers_index